In [1]:
import pandas as pd
import ast
import json
import csv
import numpy as np
import re

In [41]:
#cargamos el primer archivo csv
movies = pd.read_csv('/content/drive/MyDrive/PI01/movies_dataset_corregido.csv', sep=',')

In [42]:
#desanidamos los datos de la columna 'belongs_to_collection' y eliminamos la columna 'id' que nos queda duplicada al hacer el normalize
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else None)
movies2 = pd.json_normalize(movies["belongs_to_collection"])
movies2= movies2.drop('id', axis= 1)
movies = pd.concat([movies, movies2], axis=1,)

In [43]:
#desanidamos la columna 'genres' y ya con estos datos desanidados, procedemos a eliminar las columnas 'belong_to_collection' y 'genres'
#para que no nos quede esta informacion duplicada en nuestro dataframe

def get_genre_names(genre_list):
    genres = ast.literal_eval(genre_list)
    return ", ".join([genre["name"] for genre in genres])

movies["genres_name"] = movies["genres"].apply(get_genre_names)

movies.drop(columns=['belongs_to_collection',"genres"], inplace=True)


In [44]:
#desanidamos la columna 'production_countries' y extraemos solo el valor de 'name',
#una vez obtenida la informacion necesaria, se realiza la eliminacion de la columna 'production_countries'

def get_country_names(country_list):
    countries = ast.literal_eval(country_list)
    return ", ".join([country["name"] for country in countries])


movies["countries"] = movies["production_countries"].apply(get_country_names)

movies.drop(columns=['production_countries'], inplace=True)

In [45]:
#desanidamos la columna 'production_companies' y extraemos solo el valor de productor["name"],
#una vez obtenida la informacion necesaria, se realiza la eliminacion de la columna 'production_companies'

def get_production_names(production_list):
    productions = ast.literal_eval(production_list)
    return ", ".join([productor["name"] for productor in productions])


movies["productor"] = movies["production_companies"].apply(get_production_names)

movies.drop(columns=['production_companies'], inplace=True)

In [46]:
movies.head(n=2)

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,title,video,vote_average,vote_count,name,poster_path,backdrop_path,genres_name,countries,productor
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,Toy Story,False,7.7,5415,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"Animation, Comedy, Family",United States of America,Pixar Animation Studios
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,Jumanji,False,6.9,2413,NaN,NaN,NaN,"Adventure, Fantasy, Family",United States of America,"TriStar Pictures, Teitler Film, Interscope Com..."


In [47]:
#cargamos el segundo dataset
df = pd.read_csv('/content/drive/MyDrive/PI01/credits.csv')

In [48]:
df.head(n=2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [49]:
#desanidamos la columna 'crew' y extraemos solo el valor de 'name' de los campos filtrados por Job=Director,
#una vez obtenida la informacion necesaria, se realiza la eliminacion de las columnas ''crew' y 'cast' dejando asi solo como valor relevante el nombre del director
def get_director_names(director_list):
    crew = ast.literal_eval(director_list)
    list_directors=[]
    for job in crew:
        if job['job']== 'Director':
            list_directors.append(job['name'])
    return list_directors

df["directors"] = df["crew"].apply(lambda x: get_director_names(x) if pd.notnull(x) else None)
df['directors'] = df['directors'].astype(str)
df['directors'] = df['directors'].str.strip("[]'")
df["directors"] = df["directors"].apply(lambda x: re.sub(r'(?<=\w)([A-Z])', r' \1', x))
df['directors'] = df['directors'].astype(str)
df['directors'] = df['directors'].fillna(' ')

df = df.drop(columns=['crew','cast'])

In [50]:
df.head(n=2)

,id,directors
0,862,John Lasseter
1,8844,Joe Johnston


In [51]:
#se convierten a enteros los valores de la columna 'id' y al resultado se le eliminan los valores null
df['id'] = df['id'].astype(int)
movies = movies.dropna(how='all')
movies['id']=movies['id'].astype(int)

In [52]:
# se usa la funcion merge de la libreria pandas para unir los dos dataframe transformados por medio de las columnas 'id'
movies = pd.merge(movies,df, how='inner', left_on=['id'], right_on=['id'])
movies.head(n=2)

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,video,vote_average,vote_count,name,poster_path,backdrop_path,genres_name,countries,productor,directors
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,False,7.7,5415,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"Animation, Comedy, Family",United States of America,Pixar Animation Studios,John Lasseter
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,False,6.9,2413,NaN,NaN,NaN,"Adventure, Fantasy, Family",United States of America,"TriStar Pictures, Teitler Film, Interscope Com...",Joe Johnston
2,False,0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,False,6.5,92,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,"Romance, Comedy",United States of America,"Warner Bros., Lancaster Gate",Howard Deutch
3,False,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,False,6.1,34,NaN,NaN,NaN,"Comedy, Drama, Romance",United States of America,Twentieth Century Fox Film Corporation,Forest Whitaker
4,False,0,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,False,5.7,173,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,Comedy,United States of America,"Sandollar Productions, Touchstone Pictures",Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45533,False,0,http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,...,False,4.0,1,NaN,NaN,NaN,"Drama, Family",Iran,,Hamid Nematollah
45534,False,0,NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,...,False,9.0,3,NaN,NaN,NaN,Drama,Philippines,Sine Olivia,Lav Diaz
45535,False,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,False,3.8,6,NaN,NaN,NaN,"Action, Drama, Thriller",United States of America,American World Pictures,Mark L. Lester
45536,False,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,...,False,0.0,0,NaN,NaN,NaN,,Russia,Yermoliev,Yakov Protazanov


In [53]:
#se cambian los valores null por 0 en las columnas 'revenue' y 'budget'
# se le da formato fecha a la columna 'release_date' y se eliminan nulls
movies['revenue'] = movies['revenue'].fillna(0)
movies['budget'] = movies['budget'].fillna(0)

movies['release_date'] = pd.to_datetime(movies['release_date'], format='%Y-%m-%d')

movies = movies.dropna(subset=['release_date'])

In [54]:
# se crea una nueva columna 'year' extrayendo el año de la columna 'release_date'
movies['release_year'] = movies['release_date'].dt.year

<ipython-input-54-25b41806a60c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['release_year'] = movies['release_date'].dt.year



Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.


In [55]:
# se crea la columna 'return', dividiendo entre sí, los campos 'revenue' y 'budget'
#se reemplazan los null por 0
movies['return'] = movies['revenue'] / movies['budget']
movies['return'] = movies['return'].fillna(0)
movies['return'].replace([np.inf], 0, inplace=True)

<ipython-input-55-aa1481a1c9e2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['return'] = movies['revenue'] / movies['budget']
<ipython-input-55-aa1481a1c9e2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['return'] = movies['return'].fillna(0)
<ipython-input-55-aa1481a1c9e2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

In [56]:
#eliminamos las columnas que no aportan a nuestro analisis
movies = movies.drop(columns=['video','imdb_id','adult','original_title','poster_path', 'homepage'])

In [57]:
#exportamos nuestro dataframe como archivo CSV a google drive
movies.to_csv('/content/drive/MyDrive/PI01/Movies_API_Dataset.csv', sep=',', encoding='utf-8')